In [ ]:
import pandas as pd
from pathlib import Path
batch = pd.read_csv('sans-batchfile.csv')
batch

def get_filenames_from_batch_index(batch, index, directory = 'data'):
    batch = batch.where(pd.notna(batch), None)
    directory_path = Path(directory)
    row_list = [None if pd.isna(x) else x for x in batch.iloc[index].tolist()]
    print(row_list)
    x = [None if fname is None else (directory_path / fname).with_suffix('.mpa') for fname in row_list]
    sample_scatter_file, sample_transmission_file, can_scatter_file,can_transmission_file,direct_file,background_file = tuple(x)
    return sample_scatter_file, sample_transmission_file, can_scatter_file,can_transmission_file,direct_file,background_file

sample_scatter_file, sample_transmission_file, can_scatter_file,can_transmission_file,direct_file,background_file = get_filenames_from_batch_index(batch,3)
efficiency_file = "pixel-efficiency.txt.gz"


In [ ]:
from load_RIDSANS import *
ws_sample, ws_direct, mon, ws_pixel_adj = load_RIDSANS(
    sample_scatter_file,
    sample_transmission_file,
    can_scatter_file,
    can_transmission_file,
    direct_file,
    background_file,
    efficiency_file,
)

In [ ]:

# i = 4

background_file = f"data/old-data/09_07_24_backG_3600s_reactor_on_Fish_on.mpa"
efficiency_file = "pixel-efficiency.txt.gz"

# i_vals = list(range(1,5))
i_vals = [3]
for i in i_vals:
    sample_scatter_file = f"data/sample1_Q{i}.mpa"
    sample_transmission_file = f"data/sample_transmission_Q{i}.mpa"
    can_scatter_file = f"data/sample_empty_cuvette_Q{i}.mpa"
    # I think this translates to direct, not sure
    direct_file = f"data/no_cuvette_transmission_Q{i}.mpa"

    ws_sample, ws_direct, mon, ws_pixel_adj = load_RIDSANS(
        sample_scatter_file,
        sample_transmission_file,
        can_scatter_file,
        None,
        direct_file,
        background_file,
        efficiency_file,
    )

In [ ]:
from reduce_RIDSANS import *
from mantid.api import AnalysisDataService as ADS


for i in i_vals:
    ws_sample = ADS.retrieve(f"data/sample1_Q{i}.mpa")
    ws_direct = ADS.retrieve(f"data/no_cuvette_transmission_Q{i}.mpa")
    ws_pixel_adj = ADS.retrieve("PixelAdj")

    reduction_setup_RIDSANS(ws_sample, ws_direct,ROI=0.025)

    reduced_ws_1D = reduce_RIDSANS_1D(ws_sample, ws_pixel_adj)

    Q_axis = np.array(reduced_ws_1D.dataX(0))[:-1]
    IQ_array = reduced_ws_1D.dataY(0)
    dIQ_array = reduced_ws_1D.dataE(0)
    # plt.plot(Q_axis, IQ_array,label=f"Q{i}")
    plt.errorbar(Q_axis, IQ_array, yerr=dIQ_array, fmt='o', capsize=5, label=f"Q{i}")
plt.xlabel(r'$Q$ [Å$^{-1}$]')
plt.ylabel(r'$I(Q)$ [a.u.]')
plt.yscale('log')
plt.legend()
plt.grid()
plt.show()

Porod's law: for small enough $Q$, 
$$I(Q) \propto Q^{-4}$$

In [ ]:
from reduce_RIDSANS import *
from mantid.api import AnalysisDataService as ADS

for i in i_vals:
    ws_sample = ADS.retrieve(f"data/sample1_Q{i}.mpa")
    ws_direct = ADS.retrieve(f"data/no_cuvette_transmission_Q{i}.mpa")
    ws_pixel_adj = ADS.retrieve("PixelAdj")

    reduction_setup_RIDSANS(ws_sample, ws_direct,ROI=0.025)

    reduced_ws_2D = reduce_RIDSANS_2D(ws_sample, ws_pixel_adj)

    Q_axis = np.array(reduced_ws_2D.dataX(0))
    N_Q_bins = len(Q_axis) - 1
    Q_array = np.zeros((N_Q_bins, N_Q_bins))
    for i in range(N_Q_bins):
        Q_array[i] = reduced_ws_2D.dataY(i)
    # Q_axis, Q_array
    from matplotlib.colors import LogNorm
    # print(Q_array)
    extent = [Q_axis[0], Q_axis[-1], Q_axis[0], Q_axis[-1]]
    plt.imshow(Q_array, cmap='viridis', extent=extent, norm=LogNorm(),aspect='auto')
    plt.colorbar()  # Add a colorbar to show the color scale
    plt.xlabel(r'$Q_x$ [Å$^{-1}$]')
    plt.ylabel(r'$Q_y$ [Å$^{-1}$]')
    plt.xticks(fontsize=8)
    plt.axvline(0, linestyle='--', color='red')
    plt.axhline(0, linestyle='--', color='red')
    plt.yticks(fontsize=8)
    plt.show()